# Polish

## prompt

In [8]:
system_prompt = '''
你是一名阿拉伯语专家，我将向你提供一段阿拉伯语报纸文本，其中可能存在较多缺漏、错误。
这段文本由OCR识别得到，所以你应该首先考虑字形识别的错误，以及由于图片模糊导致的文字缺失。
在考虑了这些因素的前提下理解文本。你不需要输出修正错误后的文本，只需按要求回答问题。

问题：这段文本中描述的内容对埃及政府的态度是？
1.非常满意
2.相对满意
3.中立
4.相对不满
5.非常不满

你的回复应当遵守以下格式，在此之外不要有其它任何解释的陈述：
“态度：（编号及其内容）”

'''

print(system_prompt)


你是一名阿拉伯语专家，我将向你提供一段阿拉伯语报纸文本，其中可能存在较多缺漏、错误。
这段文本由OCR识别得到，所以你应该首先考虑字形识别的错误，以及由于图片模糊导致的文字缺失。
在考虑了这些因素的前提下理解文本。你不需要输出修正错误后的文本，只需按要求回答问题。

问题：这段文本中描述的内容对埃及政府的态度是？
1.非常满意
2.相对满意
3.中立
4.相对不满
5.非常不满

你的回复应当遵守以下格式，在此之外不要有其它任何解释的陈述：
“态度：（编号及其内容）”




## Load data

In [ ]:
from datasets import load_dataset, DatasetDict, Dataset
from huggingface_hub import login

# 输入你的 Hugging Face API token

ds = load_dataset("Adelante/Al-Ahram-kw-containing-gt3")

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\黄柏喻\.cache\huggingface\token
Login successful


C:\Users\黄柏喻\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Generating train split:   0%|          | 0/28240 [00:00<?, ? examples/s]

In [64]:
ds

DatasetDict({
    train: Dataset({
        features: ['year', 'month', 'date', 'page', 'slice', 'text', 'containing'],
        num_rows: 28240
    })
})

## API

Only small batch

In [4]:

import asyncio
from datasets import load_dataset
import json
import openai
from tqdm.asyncio import tqdm

# 加载数据集
dataset = ds['train']

dataset

Dataset({
    features: ['year', 'month', 'date', 'page', 'slice', 'text', 'containing'],
    num_rows: 28240
})

In [50]:
# take several to try

batch = [dataset[idx] for idx in range(len(dataset))]
small_batch = batch[2000:]
print(type(small_batch))

<class 'list'>


In [51]:
async def async_query_openai(query_message):
    from openai import AsyncOpenAI
    client = AsyncOpenAI(
        base_url="https://api.deepseek.com",
        api_key=""
    )

    
    response = await client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {
                    "role": "user",
                    "content": query_message # 输入给他的东西
            }
        ],
        max_tokens=4096,
        temperature=0.01,
        # 下面的都是默认参数没动过
        top_p=0.7,
        # top_k=50,
        frequency_penalty=1,
        # stop=["<|eot_id|>"],
        stream=False
    
    )

    if not response:
        return 'null'
    return response.choices[0].message.content



# 处理单个文本的函数
async def process_text(text):
    prompt = system_prompt + text
    return await async_query_openai(prompt)

# 将结果写入 .jsonl 文件的函数
def write_results_to_jsonl(rawtexts, results, filename):
    with open(filename, 'a', encoding='utf-8') as f:  # 注意改为'a'模式，追加到文件中
        for raw, result in zip(rawtexts, results):
            entry = {
                "origin":raw,
                "response": result
            }
            json.dump(entry, f, ensure_ascii=False)
            f.write('\n')

# 主函数，运行 asyncio 事件循环和写入结果
async def main():
    import nest_asyncio
    nest_asyncio.apply()
    batch_size = 100
    for i in tqdm(range(0, len(small_batch), batch_size)):
        current_batch = small_batch[i:i+batch_size]
        print(f"Processing batch {i // batch_size + 1}...")
        results = await process_all_texts(current_batch)
        write_results_to_jsonl(current_batch, results, 'results.jsonl')
        print(f"Batch {i // batch_size + 1} written to results.jsonl")
        


# 处理所有文本的函数，调整为处理一批文本
async def process_all_texts(onebatch):
    semaphore = asyncio.Semaphore(10000)
    async def process_one_text(text):
        async with semaphore:
            return await process_text(text)
    
    tasks = []
    for idx in range(len(onebatch)):
        text = "所提供给你的阿拉伯语报纸文本是：\n"+onebatch[idx]['text']
        tasks.append(process_one_text(text))
    results = await asyncio.gather(*tasks)  # 使用 gather 等待所有任务完成并保持顺序
    return results

In [52]:
await main()


  0%|                                                                                          | 0/263 [00:00<?, ?it/s]

Processing batch 1...



  0%|▎                                                                               | 1/263 [00:15<1:08:03, 15.59s/it]

Batch 1 written to results.jsonl
Processing batch 2...



  1%|▌                                                                                 | 2/263 [00:25<52:49, 12.15s/it]

...

Batch 261 written to results.jsonl
Processing batch 262...



100%|███████████████████████████████████████████████████████████████████████████████▋| 262/263 [40:32<00:09,  9.29s/it]

Batch 262 written to results.jsonl
Processing batch 263...


100%|████████████████████████████████████████████████████████████████████████████████| 263/263 [40:38<00:00,  9.27s/it]

Batch 263 written to results.jsonl


## Check result

In [66]:
import jsonlines
 
file_jsonl_path = "results.jsonl"

extract_results = []

with open(file_jsonl_path, encoding='utf-8') as file:
    for onetest in jsonlines.Reader(file):
        extract_results.append(onetest['response'])

In [67]:
len(extract_results)

28240

In [69]:
extract_results[0]

'态度：2.相对满意'

In [70]:
import re
pattern = re.compile(r'\d+')
cleansed = [pattern.search(s).group() if pattern.search(s) else None for s in extract_results]
cleansed[0]

'2'

In [101]:
for label in range(1,6):
    print(labels[str(label)],"{:.2f}".format(cleansed.count(str(label)) * 100 / len(cleansed)),'\t%')

非常满意 0.14 	%
相对满意 13.08 	%
中立    2.95 	%
相对不满 75.73 	%
非常不满 8.10 	%


In [100]:
labels = {
    '1':"非常满意",
    '2':"相对满意",
    '3':"中立   ",
    '4':"相对不满",
    '5':"非常不满"
}

In [77]:
sentiment = [labels[idx] for idx in cleansed]

In [79]:
dataset = dataset.add_column('sentiment_label_numerical', cleansed)
dataset = dataset.add_column('sentiment_label', sentiment)

In [80]:
dataset

Dataset({
    features: ['year', 'month', 'date', 'page', 'slice', 'text', 'containing', 'sentiment_label_numerical', 'sentiment_label'],
    num_rows: 28240
})

## Upload

In [81]:
dataset.push_to_hub('sentiment_analysis_v1')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/29 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Adelante/sentiment_analysis_v1/commit/d645168d5cd07f98c30af5f505807e4bf402fe1b', commit_message='Upload dataset', commit_description='', oid='d645168d5cd07f98c30af5f505807e4bf402fe1b', pr_url=None, pr_revision=None, pr_num=None)